# Archive in Sciencebase


In [1]:
from loguru import logger
from pathlib import Path
from mth5.mth5 import MTH5
from mth5.clients import MakeMTH5

from mt_metadata.timeseries import Survey, Station, Run, Electric, Magnetic

from mtpy import MT

## Set paths

Set the data directory, here I have all stations under one folder.

In [2]:
data_directory = Path(r"c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021")

# Calibration path
calibration_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\MTData\antenna_20190411.cal")

# processed transfer functions
edi_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\MTData\transfer_function_archive\Clearlake")

# archive path
archive_path = data_directory.joinpath("archive")
archive_path.mkdir(exist_ok=True)

# survey ID
survey_id = "Clearlake2022"

## Survey Metadata

In [3]:
survey_metadata = Survey(id=survey_id)
survey_metadata.acquired_by.author = (
    "Jared Peacock and Mike Mitchell (U.S. Geological Survey)"
)
survey_metadata.citation_dataset.doi = r"https://doi.org/doi:10.5066/P14KAQ3M"
survey_metadata.country = "USA"
survey_metadata.datum = "WGS84"
survey_metadata.funding_source.comments = "Project Lead is Seth Burgess"
survey_metadata.funding_source.email = "sburgess@usgs.gov"
survey_metadata.funding_source.grant_id = None
survey_metadata.funding_source.name = "California Volcano Observatory"
survey_metadata.funding_source.organization = "U.S. Geological Survey"
survey_metadata.funding_source.url = "https://www.usgs.gov/observatories/calvo"
survey_metadata.geographic_name = "Clear Lake, northern California"
survey_metadata.name = "Clear Lake 2022"
survey_metadata.project = "Understanding the Clear Lake Volcanic Field"
survey_metadata.project_lead.author = "Jared Peacock"
survey_metadata.project_lead.email = "jpeacock@usgs.gov"
survey_metadata.project_lead.organization = "U.S. Geological Survey"
survey_metadata.release_license = "CC0-1.0"
survey_metadata.summary = (
    "The project is funded by the California Volcano Observatory to understand the magmatic system of the Clear Lake Volcanic "
    "Field using magnetotelluric data to image the subsurface in 3D."
)
survey_metadata.northwest_corner.latitude = 39.322
survey_metadata.northwest_corner.longitude = -123.200 
survey_metadata.southeast_corner.latitude = 38.545
survey_metadata.southeast_corner.longitude = -122.360
survey_metadata.time_period.start = "2021-04-06T18:19:58+00:00"
survey_metadata.time_period.end = "2022-11-04T16:32:00+00:00"

## Station Metadata

In [4]:
station_metadata = Station()
station_metadata.to_dict(single=True, required=False)
station_metadata.acquired_by.author = "Jared Peacock, Mike Mitchell, and Seth Burgess"
station_metadata.acquired_by.comments = None
station_metadata.acquired_by.organization = "U.S. Geological Survey"
station_metadata.channel_layout = "L"
station_metadata.comments = None
station_metadata.data_type = "BBMT"
station_metadata.location.declination.comments = "from https://ngdc.noaa.gov/geomag/calculators/magcalc.shtml#declination"
station_metadata.location.declination.model = "WMM-2021"
station_metadata.location.declination.value = 12.5
station_metadata.orientation.method = "compass"
station_metadata.orientation.reference_frame = "geomagnetic"
station_metadata.provenance.comments = "Time series converted from Zen format to MTH5"
station_metadata.provenance.software.author = "Jared Peacock"
station_metadata.provenance.software.name = "MTH5"
station_metadata.provenance.software.version = "0.4.9"
station_metadata.provenance.submitter.author = "Jared Peacock"
station_metadata.provenance.submitter.email = "jpeacock@usgs"
station_metadata.provenance.submitter.organization = "U.S. Geological Survey"
station_metadata.location.state = "California"

## Run Metadata

In [10]:
run_metadata = Run()
run_metadata.data_logger.firmware.author = "Zonge International"
run_metadata.data_logger.firmware.name = "ZEN"
run_metadata.data_logger.firmware.version = "5357"
run_metadata.data_logger.manufacturer = "Zonge International"
run_metadata.data_logger.model = "ZEN"
run_metadata.data_logger.id = "ZEN046"
run_metadata.data_logger.power_source.comments = "rechargable lithium batteries"
run_metadata.data_logger.power_source.id = None
run_metadata.data_logger.power_source.type = "Li 30 Amp-hr"
run_metadata.data_logger.power_source.voltage.end = 15.3
run_metadata.data_logger.power_source.voltage.start = 17.0
run_metadata.data_logger.timing_system.comments = "internal clock updated by GPS timing"
run_metadata.data_logger.timing_system.drift = 0.0
run_metadata.data_logger.timing_system.type = "GPS lock"
run_metadata.data_logger.timing_system.uncertainty = 0.0
run_metadata.data_logger.type = "MT"
run_metadata.metadata_by.author = "Jared Peacock"
run_metadata.metadata_by.comments = "Most pulled from Z3D files, the rest from written field notes."
run_metadata.metadata_by.organization = "U.S. Geological Survey"

## Electric Channel Metadata

In [6]:
electric_metadata = Electric()
electric_metadata.negative.manufacturer = "Borin"
electric_metadata.negative.model = "Stelth1"
electric_metadata.negative.type = "Ag-AgCl"
electric_metadata.positive.manufacturer = "Borin"
electric_metadata.positive.model = "Stelth1"
electric_metadata.positive.type = "Ag-AgCl"
electric_metadata.type = "electric"
electric_metadata.units = "digital counts"

## Magnetic Channel Metadata
Already updated from Z3D

## Create Station MTH5s

1. Loop over each folder in the directory, make sure that it is a station
2. Save transfer function 
3. Move MTH5 to archive directory


In [11]:
# for station_folder in [
#     station_folder
#     for station_folder in data_directory.iterdir()
#     if station_folder.is_dir()
# ]:
station_folders = [Path(r"c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\cl060")]
for station_folder in station_folders:
    if station_folder.name.startswith("cl"):
        station_name = station_folder.name
        save_path = archive_path.joinpath(f"{station_name}.h5")
        if not save_path.exists():
            new_mth5 = MakeMTH5.from_zen(
                station_folder,
                sample_rates=[4096, 256],
                calibration_path=calibration_path,
                survey_id=survey_id,
                combine=True,
                **{"save_path": save_path},
            )
            logger.info(f"Created {new_mth5.name}")
            edi_fn = edi_path.joinpath(f"USGS-GMEG.2022.{station_name}.edi")
            if edi_fn.exists():
                mt_obj = MT()
                mt_obj.read(edi_fn)
                mt_obj.survey = survey_id
                mt_obj.station = station_name
                mt_obj.tf_id = mt_obj.station
                with MTH5() as m:
                    m.open_mth5(new_mth5)
                    m.add_transfer_function(mt_obj)
                logger.info(f"Added TF {mt_obj.station} to MTH5 {new_mth5.name}")
            else:
                logger.warning(f"Could not find transfer function for {station_name}")
        else:
            with MTH5() as m:
                m = m.open_mth5(save_path)
                ch_summary = m.channel_summary.to_dataframe()

                # update survey metadata
                survey_group = m.get_survey(ch_summary.survey.unique()[0])
                survey_group.metadata.update(survey_metadata)
                survey_group.write_metadata()

                # update station metadata
                station_group = survey_group.stations_group.get_station(ch_summary.station.unique()[0])
                station_group.metadata.update(station_metadata)
                station_group.write_metadata()

                # update runs
                for row in m.run_summary.itertuples():
                    run_group = m.from_reference(row.run_hdf5_reference)
                    run_group.metadata.update(run_metadata)
                    run_group.write_metadata()

                for row in ch_summary.itertuples():
                    if row.component in ["ex", "ey"]:
                        ch = m.from_reference(row.hdf5_reference)
                        if ch.metadata.type in ["electric"]:
                            ch.metadata.update(electric_metadata)
                            ch.write_metadata()
            logger.info(f"Updated metadata for {save_path.name}")
                
        

25:03:11T14:57:27 | INFO | line:769 |mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl060.h5
25:03:11T14:57:27 | INFO | line:61 |__main__ | <module> | Updated metadata for cl060.h5
